In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Set up parameters

In [2]:
dataset = 'sms'
trainingSection = 9

trainDataDir = '/Users/SamZhang/Documents/Capstone/dataset/' + dataset + '/train'

In [3]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .2, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", trainDataDir + "/spam/" + str(trainingSection) + '/' + dataset + "_train.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", trainDataDir + "/ham/" + str(trainingSection) + '/' + dataset + "_train.ham", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 40, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 50, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [4]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_EVERY=50
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=50
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/train/ham/9/sms_train.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=40
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/train/spam/9/sms_train.spam



# Process data

In [5]:
def processData():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    print(x_text[0])

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
    return x_train, y_train, x_dev, y_dev, vocab_processor

In [6]:
x_train, y_train, x_dev, y_dev, vocab_processor = processData()

Loading data...
free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question \( std txt rate \) t c 's apply 08452810075over18 's
Vocabulary Size: 8329
Train/Dev split: 4015/1003


# Building CNN

In [7]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnnmodel", str(trainingSection)))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-03-24T23:45:57.855019: step 93, loss 0.405636, acc 0.9375
2018-03-24T23:45:59.135790: step 94, loss 0.261327, acc 0.960938
2018-03-24T23:46:00.409862: step 95, loss 0.234004, acc 0.96875
2018-03-24T23:46:00.913277: step 96, loss 0.424342, acc 0.93617
2018-03-24T23:46:02.197266: step 97, loss 0.236675, acc 0.953125
2018-03-24T23:46:03.448166: step 98, loss 0.212948, acc 0.976562
2018-03-24T23:46:04.757600: step 99, loss 0.338097, acc 0.9375
2018-03-24T23:46:06.049658: step 100, loss 0.221785, acc 0.984375

Evaluation:
2018-03-24T23:46:09.648881: step 100, loss 0.241753, acc 0.976072

Saved model checkpoint to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/9/checkpoints/model-100

2018-03-24T23:46:11.298381: step 101, loss 0.304848, acc 0.953125
2018-03-24T23:46:12.699866: step 102, loss 0.21859, acc 0.96875
2018-03-24T23:46:14.040606: step 103, loss 0.225417, acc 0.976562
2018-03-24T23:46:15.387891: step 104, loss 0.223786, acc 0.976562
2018-03-24T23:46:16.745303: step 105

2018-03-24T23:48:49.253050: step 211, loss 0.152739, acc 0.992188
2018-03-24T23:48:50.717991: step 212, loss 0.172618, acc 0.984375
2018-03-24T23:48:52.157394: step 213, loss 0.164128, acc 0.992188
2018-03-24T23:48:53.557570: step 214, loss 0.1557, acc 0.992188
2018-03-24T23:48:54.999811: step 215, loss 0.14963, acc 0.992188
2018-03-24T23:48:56.510192: step 216, loss 0.151243, acc 0.992188
2018-03-24T23:48:58.082469: step 217, loss 0.158456, acc 0.984375
2018-03-24T23:48:59.584826: step 218, loss 0.138427, acc 1
2018-03-24T23:49:01.105911: step 219, loss 0.250942, acc 0.96875
2018-03-24T23:49:02.539201: step 220, loss 0.137984, acc 1
2018-03-24T23:49:03.979014: step 221, loss 0.149076, acc 0.984375
2018-03-24T23:49:05.447007: step 222, loss 0.175291, acc 0.984375
2018-03-24T23:49:06.878472: step 223, loss 0.135023, acc 1
2018-03-24T23:49:07.456174: step 224, loss 0.126711, acc 1
2018-03-24T23:49:08.900778: step 225, loss 0.193423, acc 0.96875
2018-03-24T23:49:10.399528: step 226, loss 

2018-03-24T23:51:52.750565: step 335, loss 0.104654, acc 1
2018-03-24T23:51:54.161392: step 336, loss 0.114674, acc 0.992188
2018-03-24T23:51:55.605860: step 337, loss 0.155392, acc 0.984375
2018-03-24T23:51:57.031420: step 338, loss 0.105027, acc 0.992188
2018-03-24T23:51:58.475618: step 339, loss 0.105424, acc 0.992188
2018-03-24T23:51:59.925058: step 340, loss 0.125963, acc 0.992188
2018-03-24T23:52:01.325110: step 341, loss 0.097188, acc 1
2018-03-24T23:52:02.708959: step 342, loss 0.105961, acc 0.992188
2018-03-24T23:52:04.080418: step 343, loss 0.102101, acc 0.992188
2018-03-24T23:52:05.481032: step 344, loss 0.140011, acc 0.984375
2018-03-24T23:52:06.857952: step 345, loss 0.102184, acc 0.992188
2018-03-24T23:52:08.289555: step 346, loss 0.093546, acc 1
2018-03-24T23:52:09.702097: step 347, loss 0.128583, acc 0.984375
2018-03-24T23:52:11.120179: step 348, loss 0.0963412, acc 1
2018-03-24T23:52:12.524519: step 349, loss 0.102474, acc 1
2018-03-24T23:52:13.971606: step 350, loss 0

2018-03-24T23:55:02.449222: step 456, loss 0.101612, acc 0.992188
2018-03-24T23:55:03.871154: step 457, loss 0.0667234, acc 1
2018-03-24T23:55:05.306472: step 458, loss 0.0683279, acc 1
2018-03-24T23:55:06.735177: step 459, loss 0.0795865, acc 0.992188
2018-03-24T23:55:08.168222: step 460, loss 0.0687777, acc 1
2018-03-24T23:55:09.606675: step 461, loss 0.0841455, acc 0.992188
2018-03-24T23:55:11.088712: step 462, loss 0.0692338, acc 1
2018-03-24T23:55:12.669963: step 463, loss 0.0933503, acc 0.984375
2018-03-24T23:55:14.217849: step 464, loss 0.064266, acc 1
2018-03-24T23:55:15.744561: step 465, loss 0.0735775, acc 1
2018-03-24T23:55:17.285070: step 466, loss 0.0683816, acc 1
2018-03-24T23:55:18.810407: step 467, loss 0.0679728, acc 1
2018-03-24T23:55:20.560463: step 468, loss 0.0656657, acc 1
2018-03-24T23:55:22.055557: step 469, loss 0.0760533, acc 1
2018-03-24T23:55:23.527490: step 470, loss 0.0745917, acc 0.992188
2018-03-24T23:55:25.010365: step 471, loss 0.0776616, acc 1
2018-03

2018-03-24T23:58:03.536890: step 583, loss 0.0490652, acc 1
2018-03-24T23:58:04.863835: step 584, loss 0.0556186, acc 1
2018-03-24T23:58:06.254535: step 585, loss 0.047074, acc 1
2018-03-24T23:58:07.641371: step 586, loss 0.0468399, acc 1
2018-03-24T23:58:09.013948: step 587, loss 0.0464365, acc 1
2018-03-24T23:58:10.395982: step 588, loss 0.051115, acc 0.992188
2018-03-24T23:58:11.795335: step 589, loss 0.0486601, acc 1
2018-03-24T23:58:13.169840: step 590, loss 0.049056, acc 1
2018-03-24T23:58:14.554982: step 591, loss 0.0505049, acc 1
2018-03-24T23:58:16.023597: step 592, loss 0.0479089, acc 1
2018-03-24T23:58:17.396569: step 593, loss 0.0425799, acc 1
2018-03-24T23:58:18.770237: step 594, loss 0.0438996, acc 1
2018-03-24T23:58:20.124846: step 595, loss 0.0436092, acc 1
2018-03-24T23:58:21.491368: step 596, loss 0.0496138, acc 0.992188
2018-03-24T23:58:22.838947: step 597, loss 0.0494702, acc 1
2018-03-24T23:58:24.184668: step 598, loss 0.0475006, acc 1
2018-03-24T23:58:25.536489: s

2018-03-25T00:01:10.977965: step 710, loss 0.030653, acc 1
2018-03-25T00:01:12.471227: step 711, loss 0.03557, acc 1
2018-03-25T00:01:13.960413: step 712, loss 0.0436663, acc 0.992188
2018-03-25T00:01:15.417364: step 713, loss 0.0334332, acc 1
2018-03-25T00:01:16.933673: step 714, loss 0.0334988, acc 1
2018-03-25T00:01:18.724244: step 715, loss 0.0328226, acc 1
2018-03-25T00:01:20.365227: step 716, loss 0.0427418, acc 0.992188
2018-03-25T00:01:21.841256: step 717, loss 0.0344037, acc 1
2018-03-25T00:01:23.389478: step 718, loss 0.0327512, acc 1
2018-03-25T00:01:24.923410: step 719, loss 0.0289252, acc 1
2018-03-25T00:01:26.715642: step 720, loss 0.0330934, acc 1
2018-03-25T00:01:28.565087: step 721, loss 0.036761, acc 1
2018-03-25T00:01:30.123627: step 722, loss 0.0284597, acc 1
2018-03-25T00:01:31.893085: step 723, loss 0.0285528, acc 1
2018-03-25T00:01:33.608208: step 724, loss 0.0325185, acc 1
2018-03-25T00:01:35.440329: step 725, loss 0.0287573, acc 1
2018-03-25T00:01:37.092411: st

KeyboardInterrupt: 

# Testing model

In [8]:
from sklearn import metrics
import csv

In [9]:
testDataDir = "/Users/SamZhang/Documents/Capstone/dataset/" + dataset + "/test"

In [10]:
tf.flags.DEFINE_string("positive_test_data_file", testDataDir + "/spam/" + str(trainingSection) + '/' + dataset + "_test.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_test_data_file", testDataDir + "/ham/" + str(trainingSection) + '/' + dataset + "_test.ham", "Data source for the negative data.")

# Eval Parameters
tf.flags.DEFINE_string("checkpoint_dir", "/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/" + str(trainingSection) + "/checkpoints/", "Checkpoint directory from training run")

In [11]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_DIR=/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/9/checkpoints/
CHECKPOINT_EVERY=50
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=50
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/train/ham/9/sms_train.ham
NEGATIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/test/ham/9/sms_test.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=40
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/train/spam/9/sms_train.spam
POSITIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/sms/test/spam/9/sms_test.spam



In [12]:
x_raw, y_test = data_helpers.load_data_and_labels(FLAGS.positive_test_data_file, FLAGS.negative_test_data_file)
y_test = np.argmax(y_test, axis=1) #ham = 0, spam = 1
print(x_raw[0], y_test[0])

07732584351 rodger burns msg we tried to call you re your reply to our sms for a free nokia mobile free camcorder please call now 08000930705 for delivery tomorrow 1


In [13]:
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

In [14]:
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

In [15]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

INFO:tensorflow:Restoring parameters from /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/9/checkpoints/model-750
Total number of test examples: 556
Accuracy: 0.96223


In [16]:
# Save the evaluation to a csv
title = np.column_stack(('text', 'prediction', 'label'))
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions, y_test))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))

with open(out_path, 'w') as f:
    csv.writer(f).writerows(title)
    csv.writer(f).writerows(predictions_human_readable)

Saving evaluation to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/9/checkpoints/../prediction.csv


# Evaluation and Noise Reduction

In [17]:
import ES_interface as esi

esi.metric(dataset + '_' + str(trainingSection), out_path)

53 0 482 21
accuracy : 0.962 
precision : 1.000 
recall : 0.716 
f1 : 0.835 

52 0 482 22
accuracy : 0.960 
precision : 1.000 
recall : 0.703 
f1 : 0.825 

